In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
ed=pd.read_parquet("new_edstays_240306.parquet")

In [3]:
ed

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,result
0,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,White,AMBULANCE,ADMITTED,hos
1,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,White,AMBULANCE,ADMITTED,hos
2,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,White,AMBULANCE,ADMITTED,icu
3,10000084,23052089.0,35203156,2160-11-20 20:36:00,2160-11-21 03:20:00,M,White,WALK IN,ADMITTED,hos
4,10000108,NaN,32522732,2163-09-16 16:34:00,2163-09-16 18:13:00,M,White,WALK IN,HOME,ed
...,...,...,...,...,...,...,...,...,...,...
341520,19999784,24935234.0,37972930,2119-07-09 17:38:00,2119-07-10 00:04:00,M,Black,WALK IN,ADMITTED,hos
341521,19999784,25715748.0,34149746,2119-08-11 09:15:00,2119-08-11 13:40:00,M,Black,WALK IN,ADMITTED,hos
341522,19999784,26194817.0,35692999,2119-06-18 14:21:00,2119-06-18 21:09:29,M,Black,WALK IN,ADMITTED,hos
341523,19999828,25744818.0,32917002,2149-01-08 09:11:00,2149-01-08 18:12:00,F,White,AMBULANCE,ADMITTED,icu


In [4]:
ed_1=ed[['stay_id','intime','gender','race','arrival_transport','result']]
ed_2=ed[['stay_id','outtime','gender','race','arrival_transport','result']]

In [5]:
ed_1.rename(columns={'intime': 'time'}, inplace=True)
ed_2.rename(columns={'outtime': 'time'}, inplace=True)

/tmp/ipykernel_2449120/2198893246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_1.rename(columns={'intime': 'time'}, inplace=True)
/tmp/ipykernel_2449120/2198893246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_2.rename(columns={'outtime': 'time'}, inplace=True)


In [6]:
ed=pd.concat([ed_1,ed_2])
ed=ed.sort_values(by=['stay_id','time'])

In [7]:
ed.reset_index(drop=True,inplace=True)

In [8]:
ed

,stay_id,time,gender,race,arrival_transport,result
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos
1,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos
2,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos
3,30000038,2152-12-07 19:55:00,F,White,AMBULANCE,hos
4,30000039,2165-10-06 11:47:00,M,White,WALK IN,hos
...,...,...,...,...,...,...
683045,39999933,2128-07-20 19:49:00,M,Black,WALK IN,ed
683046,39999953,2152-06-22 14:08:00,F,Other,WALK IN,ed
683047,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed
683048,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos


In [9]:
triage=pd.read_parquet('triage.parquet')
triage

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,10000032,32952584,97.8,87.0,14.0,97.0,71.0,43.0,7,2.0,Hypotension
1,10000032,33258284,98.4,70.0,16.0,97.0,106.0,63.0,0,3.0,"Abd pain, Abdominal distention"
2,10000032,35968195,99.4,105.0,18.0,96.0,106.0,57.0,10,3.0,"n/v/d, Abd pain"
3,10000032,38112554,98.9,88.0,18.0,97.0,116.0,88.0,10,3.0,Abdominal distention
4,10000032,39399961,98.7,77.0,16.0,98.0,96.0,50.0,13,2.0,"Abdominal distention, Abd pain, LETHAGIC"
...,...,...,...,...,...,...,...,...,...,...,...
425082,19999784,37972930,98.0,91.0,16.0,99.0,148.0,90.0,5,2.0,Abnormal MRI
425083,19999828,30712109,98.1,83.0,18.0,100.0,107.0,75.0,8,2.0,"Abd pain, Wound eval"
425084,19999828,32917002,96.6,112.0,18.0,100.0,110.0,82.0,4,2.0,"Abd pain, Wound eval"
425085,19999914,32002659,99.5,81.0,10.0,100.0,93.0,55.0,0,2.0,"Altered mental status, Substance use"


In [10]:
#triage 단순화 
triage['chiefcomplaint'] = triage['chiefcomplaint'].str.lower()
triage['chiefcomplaint']= triage['chiefcomplaint'].str.replace(" ", "")

In [11]:
triage

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,10000032,32952584,97.8,87.0,14.0,97.0,71.0,43.0,7,2.0,hypotension
1,10000032,33258284,98.4,70.0,16.0,97.0,106.0,63.0,0,3.0,"abdpain,abdominaldistention"
2,10000032,35968195,99.4,105.0,18.0,96.0,106.0,57.0,10,3.0,"n/v/d,abdpain"
3,10000032,38112554,98.9,88.0,18.0,97.0,116.0,88.0,10,3.0,abdominaldistention
4,10000032,39399961,98.7,77.0,16.0,98.0,96.0,50.0,13,2.0,"abdominaldistention,abdpain,lethagic"
...,...,...,...,...,...,...,...,...,...,...,...
425082,19999784,37972930,98.0,91.0,16.0,99.0,148.0,90.0,5,2.0,abnormalmri
425083,19999828,30712109,98.1,83.0,18.0,100.0,107.0,75.0,8,2.0,"abdpain,woundeval"
425084,19999828,32917002,96.6,112.0,18.0,100.0,110.0,82.0,4,2.0,"abdpain,woundeval"
425085,19999914,32002659,99.5,81.0,10.0,100.0,93.0,55.0,0,2.0,"alteredmentalstatus,substanceuse"


In [12]:
triage['abdpain'] = triage['chiefcomplaint'].str.contains('abdpain', na=False).astype(int)
triage['transfer'] = triage['chiefcomplaint'].str.contains('transfer', na=False).astype(int)
triage['chestpain'] = triage['chiefcomplaint'].str.contains('chestpain', na=False).astype(int)
triage['pfall'] = triage['chiefcomplaint'].str.contains('pfall', na=False).astype(int)
triage['dyspnea'] = triage['chiefcomplaint'].str.contains('dyspnea', na=False).astype(int)
triage['fever'] = triage['chiefcomplaint'].str.contains('fever', na=False).astype(int)
triage['headache'] = triage['chiefcomplaint'].str.contains('headache', na=False).astype(int)
triage['backpain'] = triage['chiefcomplaint'].str.contains('backpain', na=False).astype(int)
triage['weakness'] = triage['chiefcomplaint'].str.contains('weakness', na=False).astype(int)
triage['dizziness'] = triage['chiefcomplaint'].str.contains('dizziness', na=False).astype(int)
triage['cough'] = triage['chiefcomplaint'].str.contains('cough', na=False).astype(int)
triage['woundeval'] = triage['chiefcomplaint'].str.contains('woundeval', na=False).astype(int)
triage['mvc'] = triage['chiefcomplaint'].str.contains('mvc', na=False).astype(int)
triage['etoh'] = triage['chiefcomplaint'].str.contains('etoh', na=False).astype(int)
triage['altered_mental_status'] = triage['chiefcomplaint'].str.contains('alteredmentalstatus', na=False).astype(int)

In [13]:
triage

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status
0,10000032,32952584,97.8,87.0,14.0,97.0,71.0,43.0,7,2.0,hypotension,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000032,33258284,98.4,70.0,16.0,97.0,106.0,63.0,0,3.0,"abdpain,abdominaldistention",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000032,35968195,99.4,105.0,18.0,96.0,106.0,57.0,10,3.0,"n/v/d,abdpain",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000032,38112554,98.9,88.0,18.0,97.0,116.0,88.0,10,3.0,abdominaldistention,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000032,39399961,98.7,77.0,16.0,98.0,96.0,50.0,13,2.0,"abdominaldistention,abdpain,lethagic",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425082,19999784,37972930,98.0,91.0,16.0,99.0,148.0,90.0,5,2.0,abnormalmri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
425083,19999828,30712109,98.1,83.0,18.0,100.0,107.0,75.0,8,2.0,"abdpain,woundeval",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425084,19999828,32917002,96.6,112.0,18.0,100.0,110.0,82.0,4,2.0,"abdpain,woundeval",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425085,19999914,32002659,99.5,81.0,10.0,100.0,93.0,55.0,0,2.0,"alteredmentalstatus,substanceuse",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [14]:
triage['temperature']=(triage['temperature']-32)/1.8

In [15]:
triage['temperature'] = triage['temperature'].apply(lambda x: np.nan if x < 32 or x >= 43 else x)
triage['heartrate'] = triage['heartrate'].apply(lambda x: np.nan if x <= 0 or x >= 300 else x)
triage['o2sat'] = triage['o2sat'].apply(lambda x: np.nan if x < 10 or x > 100 else x)
triage['resprate'] = triage['resprate'].apply(lambda x: np.nan if x <= 0 or x > 60 else x)
triage['sbp'] = triage['sbp'].apply(lambda x: np.nan if x <= 0 or x > 300 else x)
triage['dbp'] = triage['dbp'].apply(lambda x: np.nan if x < 10 or x > 175 else x)
triage.loc[triage['sbp'] < triage['dbp'],['sbp','dbp']]=np.nan

In [16]:
triage

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status
0,10000032,32952584,36.555556,87.0,14.0,97.0,71.0,43.0,7,2.0,hypotension,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000032,33258284,36.888889,70.0,16.0,97.0,106.0,63.0,0,3.0,"abdpain,abdominaldistention",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000032,35968195,37.444444,105.0,18.0,96.0,106.0,57.0,10,3.0,"n/v/d,abdpain",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000032,38112554,37.166667,88.0,18.0,97.0,116.0,88.0,10,3.0,abdominaldistention,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000032,39399961,37.055556,77.0,16.0,98.0,96.0,50.0,13,2.0,"abdominaldistention,abdpain,lethagic",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425082,19999784,37972930,36.666667,91.0,16.0,99.0,148.0,90.0,5,2.0,abnormalmri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
425083,19999828,30712109,36.722222,83.0,18.0,100.0,107.0,75.0,8,2.0,"abdpain,woundeval",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425084,19999828,32917002,35.888889,112.0,18.0,100.0,110.0,82.0,4,2.0,"abdpain,woundeval",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
425085,19999914,32002659,37.500000,81.0,10.0,100.0,93.0,55.0,0,2.0,"alteredmentalstatus,substanceuse",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
triage_df= triage.drop(['subject_id', 'pain','chiefcomplaint'], axis=1)

In [18]:
triage_df.to_csv("triage_240308.csv",index=False)

In [19]:
ed

,stay_id,time,gender,race,arrival_transport,result
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos
1,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos
2,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos
3,30000038,2152-12-07 19:55:00,F,White,AMBULANCE,hos
4,30000039,2165-10-06 11:47:00,M,White,WALK IN,hos
...,...,...,...,...,...,...
683045,39999933,2128-07-20 19:49:00,M,Black,WALK IN,ed
683046,39999953,2152-06-22 14:08:00,F,Other,WALK IN,ed
683047,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed
683048,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos


In [20]:
triage_df=triage_df[triage_df['stay_id'].isin(ed['stay_id'])]
triage_df=triage_df.sort_values(by=['stay_id'])
triage_df.reset_index(drop=True,inplace=True)

In [21]:
triage_df

,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status
0,30000012,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000038,36.166667,54.0,18.0,95.0,143.0,73.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,30000039,37.000000,85.0,16.0,98.0,189.0,96.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,30000055,37.444444,85.0,16.0,100.0,NaN,NaN,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,30000094,36.722222,60.0,18.0,94.0,120.0,95.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341520,39999835,36.166667,90.0,22.0,98.0,168.0,80.0,2.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
341521,39999863,36.555556,46.0,18.0,100.0,222.0,83.0,2.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
341522,39999933,37.000000,114.0,14.0,100.0,131.0,81.0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
341523,39999953,36.777778,108.0,18.0,100.0,155.0,94.0,3.0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0


In [22]:
sorted_ed = ed.sort_values(by=['stay_id', 'time'])

In [23]:
modified_ed = sorted_ed.copy()
modified_ed['time'] = pd.to_datetime(modified_ed['time'])
for index, row in tqdm(modified_ed.iterrows()):
    if index % 2 == 0:  # Odd rows
        modified_ed.at[index, 'time'] += timedelta(minutes=1)

683050it [00:42, 16138.02it/s]


In [24]:
odd_rows_time = modified_ed[modified_ed.index % 2 == 0][['stay_id','time']]

In [25]:
triage_df_1=pd.merge(triage_df,odd_rows_time,how='left',on='stay_id')

In [26]:
triage_df_1

,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status,time
0,30000012,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2126-02-14 20:23:00
1,30000038,36.166667,54.0,18.0,95.0,143.0,73.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2152-12-07 16:38:00
2,30000039,37.000000,85.0,16.0,98.0,189.0,96.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2165-10-06 11:48:00
3,30000055,37.444444,85.0,16.0,100.0,NaN,NaN,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2155-07-18 17:04:00
4,30000094,36.722222,60.0,18.0,94.0,120.0,95.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2183-09-04 16:09:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341520,39999835,36.166667,90.0,22.0,98.0,168.0,80.0,2.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2137-03-19 11:50:00
341521,39999863,36.555556,46.0,18.0,100.0,222.0,83.0,2.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2122-01-16 18:03:00
341522,39999933,37.000000,114.0,14.0,100.0,131.0,81.0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2128-07-20 13:43:00
341523,39999953,36.777778,108.0,18.0,100.0,155.0,94.0,3.0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2152-06-22 14:09:00


In [27]:
df=pd.concat([ed, triage_df_1])

In [28]:
df['time']=pd.to_datetime(df['time'])

In [29]:
df=df.sort_values(by=['stay_id', 'time'])
df.reset_index(drop=True,inplace=True)

In [30]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,abdpain,transfer,chestpain,pfall,dyspnea,fever,headache,backpain,weakness,dizziness,cough,woundeval,mvc,etoh,altered_mental_status
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:38:00,NaN,NaN,NaN,NaN,36.166667,54.0,18.0,95.0,143.0,73.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024570,39999953,2152-06-22 14:09:00,NaN,NaN,NaN,NaN,36.777778,108.0,18.0,100.0,155.0,94.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1024571,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024572,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024573,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [31]:
pyxis=pd.read_parquet('pyxis.parquet')

In [32]:
pyxis

,subject_id,stay_id,charttime,med_rn,name,gsn_rn,gsn
0,10000032,32952584,2180-07-22 17:59:00,1,Albuterol Inhaler,1,5037.0
1,10000032,32952584,2180-07-22 17:59:00,1,Albuterol Inhaler,2,28090.0
2,10000032,35968195,2180-08-05 22:29:00,1,Morphine,1,4080.0
3,10000032,35968195,2180-08-05 22:55:00,2,Donnatol (Elixir),1,4773.0
4,10000032,35968195,2180-08-05 22:55:00,3,Aluminum-Magnesium Hydrox.-Simet,1,2701.0
...,...,...,...,...,...,...,...
1586048,19999914,32002659,2158-12-24 11:52:00,2,LORazepam 2mg/1mL 1mL SYR,3,78733.0
1586049,19999987,34731548,2145-11-02 19:30:00,1,Midazolam,1,3780.0
1586050,19999987,34731548,2145-11-02 19:30:00,1,Midazolam,2,51279.0
1586051,19999987,34731548,2145-11-02 19:30:00,2,Fentanyl,1,41385.0


In [33]:
drug_info=pd.read_parquet('pyxis_drug.parquet')

In [34]:
pyxis_df=pyxis[['stay_id','charttime','name']]

In [35]:
pyxis_df_1=pd.merge(pyxis_df,drug_info,on='name',how='left')

In [36]:
pyxis_df_1=pyxis_df_1[['stay_id','charttime','change_drug']]

In [37]:
pyxis_df_1

,stay_id,charttime,change_drug
0,32952584,2180-07-22 17:59:00,albuterol
1,32952584,2180-07-22 17:59:00,albuterol
2,35968195,2180-08-05 22:29:00,morphine
3,35968195,2180-08-05 22:55:00,donnatol
4,35968195,2180-08-05 22:55:00,aluminum-magnesium-simet
...,...,...,...
1586048,32002659,2158-12-24 11:52:00,lorazepam
1586049,34731548,2145-11-02 19:30:00,midazolam
1586050,34731548,2145-11-02 19:30:00,midazolam
1586051,34731548,2145-11-02 19:30:00,fentanyl


In [38]:
pyxis_df_2=pyxis_df_1[~pyxis_df_1.duplicated()]
pyxis_df_2=pyxis_df_2[pyxis_df_2['stay_id'].isin(ed['stay_id'])]
pyxis_df_2.rename(columns={'charttime': 'time'}, inplace=True)
pyxis_df_2=pyxis_df_2.sort_values(by=['stay_id', 'time'])
pyxis_df_2.reset_index(drop=True, inplace=True)

In [39]:
pyxis_df_2

,stay_id,time,change_drug
0,30000012,2126-02-15 00:44:00,spironolactone
1,30000012,2126-02-15 00:44:00,gabapentin
2,30000012,2126-02-15 01:22:00,ceftriaxone
3,30000038,2152-12-07 18:31:00,ceftriaxone
4,30000038,2152-12-07 18:32:00,azithromycin
...,...,...,...
836583,39999933,2128-07-20 14:57:00,readi-cat 2
836584,39999933,2128-07-20 18:45:00,senna
836585,39999933,2128-07-20 18:49:00,magnesium
836586,39999961,2145-05-16 22:33:00,heparin


In [40]:
medications = ['acetaminophen', 'morphine', 'ondansetron', 'hydromorphone', 'oxycodone','lidocaine', 'ibuprofen', 'lorazepam', 'albuterol', 'ketorolac','ceftriaxone', 'ipratropium', 'vancomycin', 'bromide', 'diazepam','aspirin', 'sodium', 'magnesium', 'fentanyl', 'diphenhydramine']

In [41]:
for i in tqdm(medications):
    #print('drug_'+i)
    pyxis_df_2['drug_'+i] = pyxis_df_2['change_drug'].str.contains(i, na=False).astype(int)

  5%|▌         | 1/20 [00:00<00:02,  6.90it/s]

100%|██████████| 20/20 [00:02<00:00,  7.49it/s]


In [42]:
sum(pyxis_df_2.iloc[:,3:].sum(axis=1)==1)

485306

In [43]:
pyxis_df_2=pyxis_df_2[pyxis_df_2.iloc[:, 3:].sum(axis=1) == 1]
pyxis_df_2.reset_index(inplace=True,drop=True)

In [44]:
pyxis_df_2.drop(columns=['change_drug'], inplace=True)

/tmp/ipykernel_2449120/728501048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pyxis_df_2.drop(columns=['change_drug'], inplace=True)


In [45]:
pyxis_df_2.columns[2:]

Index(['drug_acetaminophen', 'drug_morphine', 'drug_ondansetron',
       'drug_hydromorphone', 'drug_oxycodone', 'drug_lidocaine',
       'drug_ibuprofen', 'drug_lorazepam', 'drug_albuterol', 'drug_ketorolac',
       'drug_ceftriaxone', 'drug_ipratropium', 'drug_vancomycin',
       'drug_bromide', 'drug_diazepam', 'drug_aspirin', 'drug_sodium',
       'drug_magnesium', 'drug_fentanyl', 'drug_diphenhydramine'],
      dtype='object')

In [46]:
pyxis_df_3=pyxis_df_2.groupby(['stay_id', 'time'])[pyxis_df_2.columns[2:]].max().reset_index()

In [47]:
pyxis_df_3

,stay_id,time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-15 01:22:00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,30000038,2152-12-07 18:31:00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,30000177,2143-12-28 00:03:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,30000177,2143-12-28 00:04:00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,30000202,2158-08-08 10:13:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422897,39999828,2126-12-08 15:17:00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
422898,39999835,2137-03-19 12:59:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
422899,39999933,2128-07-20 14:52:00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
422900,39999933,2128-07-20 18:49:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [48]:
 df.columns[13:]

Index(['abdpain', 'transfer', 'chestpain', 'pfall', 'dyspnea', 'fever',
       'headache', 'backpain', 'weakness', 'dizziness', 'cough', 'woundeval',
       'mvc', 'etoh', 'altered_mental_status'],
      dtype='object')

In [49]:
new_columns = ['chief_' + column for column in df.columns[13:]]
df.columns = list(df.columns[:13]) + new_columns

In [50]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:38:00,NaN,NaN,NaN,NaN,36.166667,54.0,18.0,95.0,143.0,73.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024570,39999953,2152-06-22 14:09:00,NaN,NaN,NaN,NaN,36.777778,108.0,18.0,100.0,155.0,94.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1024571,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024572,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024573,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [51]:
lasttime=df.groupby('stay_id')['time'].min().reset_index()

In [52]:
lasttime=lasttime.rename(columns={'time': 'first_time'})

In [53]:
df=pd.merge(df, lasttime,on='stay_id',how='left')

In [54]:
lasttime=df.groupby('stay_id')['time'].max().reset_index()
lasttime=lasttime.rename(columns={'time': 'last_time'})

In [55]:
df=pd.merge(df, lasttime,on='stay_id',how='left')

In [56]:
df

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00
2,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00
3,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00
4,30000038,2152-12-07 16:38:00,NaN,NaN,NaN,NaN,36.166667,54.0,18.0,95.0,143.0,73.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2152-12-07 16:37:00,2152-12-07 19:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024570,39999953,2152-06-22 14:09:00,NaN,NaN,NaN,NaN,36.777778,108.0,18.0,100.0,155.0,94.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2152-06-22 14:08:00,2152-06-22 18:13:00
1024571,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,2152-06-22 18:13:00
1024572,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00
1024573,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2145-05-16 17:16:00,2145-05-17 01:28:00


In [57]:
df_1=pd.concat([df,pyxis_df_3])

In [58]:
df_1['time']=pd.to_datetime(df_1['time'])
df_1=df_1.sort_values(by=['stay_id', 'time'])
df_1.reset_index(drop=True, inplace=True)

In [59]:
df_1

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447472,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,2152-06-22 18:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447473,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447474,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447475,39999961,2145-05-16 22:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
df_1['first_time']=df_1.groupby('stay_id')['first_time'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df_1['last_time']=df_1.groupby('stay_id')['last_time'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [61]:
df_1

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447472,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,2152-06-22 18:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447473,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447474,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447475,39999961,2145-05-16 22:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
df_1[df_1['last_time']<df_1['time']]

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
281,30001829,2175-03-12 07:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175-03-12 00:37:00,2175-03-12 03:57:58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
381,30002780,2180-02-26 12:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180-02-24 15:03:00,2180-02-24 22:47:31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
658,30004806,2196-07-20 04:47:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2196-07-19 19:04:00,2196-07-20 04:45:59,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
659,30004806,2196-07-20 07:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2196-07-19 19:04:00,2196-07-20 04:45:59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
660,30004806,2196-07-20 08:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2196-07-19 19:04:00,2196-07-20 04:45:59,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446991,39996613,2151-09-07 04:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2151-09-06 23:59:00,2151-09-07 02:53:15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1446992,39996613,2151-09-07 05:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2151-09-06 23:59:00,2151-09-07 02:53:15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1447130,39997465,2185-11-21 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2185-11-21 15:13:00,2185-11-21 17:24:05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1447344,39998942,2129-04-22 19:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2129-04-22 08:24:00,2129-04-22 12:25:03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [63]:
df_2=df_1[df_1['last_time']>=df_1['time']]

In [64]:
df_2

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447472,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,2152-06-22 18:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447473,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447474,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447475,39999961,2145-05-16 22:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#df_2['first_time']=pd.to_datetime(df_2['first_time'])
df_3=df_2[df_2['first_time']<=df_2['time']]

In [66]:
df_3

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447472,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,2152-06-22 18:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447473,39999961,2145-05-16 17:16:00,F,Black,WALK IN,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447474,39999961,2145-05-16 17:17:00,NaN,NaN,NaN,NaN,37.388889,119.0,22.0,NaN,132.0,74.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2145-05-16 17:16:00,2145-05-17 01:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447475,39999961,2145-05-16 22:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145-05-16 17:16:00,2145-05-17 01:28:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
diag=pd.read_parquet('diagnosis.parquet')

In [68]:
diag_df=diag[['stay_id','icd_code','icd_version']]
diag_df

,stay_id,icd_code,icd_version
0,32952584,4589,9
1,32952584,07070,9
2,32952584,V08,9
3,33258284,5728,9
4,33258284,78959,9
...,...,...,...
899045,30712109,K632,10
899046,32917002,E1110,10
899047,32917002,Z7984,10
899048,32002659,R4182,10


In [69]:
diag_df['icd_code'].value_counts()

icd_code
4019       26816
I10        21264
R079       13016
78650      12398
25000      12026
           ...  
5834           1
S62366A        1
S62639A        1
M4200          1
80425          1
Name: count, Length: 13199, dtype: int64

In [70]:
hypertension_uncomplicated_9=['401']
hypertension_complicated_9=['402','403','404','405']
chf_9=['39891', '40201', '40211','40291', '40401', '40403', '40411', '40413', '40491', '40493', '4254','4255','4256','4257','4258','4259','428']
cardiac_9=['4260', '42613', '4267', '4269', '42610', '42612', '4270','4271','4272','4273','4274','4276','4277','4278','4279','7850', '99601', '99604', 'V450', 'V533']
valvular_disease_9=['0932','394','395','396','397','424','7463','7464','7465','7466', 'V422', 'V433']
Pulmonary_circulation_9=['4150', '4151', '416', '4170', '4178', '4179']
Peripheral_vascular_9=['0930', '4373', '440', '441', '4431','4432',    '4433',    '4434',    '4435',    '4436',    '4437',    '4438',    '4439', '4471', '5571', '5579', 'V434']
Paralysis_9=['3341', '342', '343', '3440','3441','3442','3443','3444','3445','3446', '3449' ]
Other_neurological_disorders_9=['3319', '3320', '3321', '3334', '3335', '33392', '334','335', '3362', '340', '341', '345', '3481', '3483', '7803', '7843']
Chronic_pulmonary_disease_9=['4168', '4169','490', '491', '492', '493', '494', '495', '496', '497', '498', '499', '500', '501', '502', '503', '504', '505','5064', '5081', '5088']
Diabetes_uncomplicated_9=['2500','2501','2502','2503']
Diabetes_complicated_9=['2504','2505','2506','2507','2508','2509']
Hypothyroidism_9=['2409', '243', '244', '2461', '2468']
Renal_failure_9=['40301', '40311', '40391', '40402', '40403', '40412', '40413', '40492', '40493', '585', '586', '5880', 'V420', 'V451', 'V56' ]
Liver_disease_9=['07022', '07023', '07032', '07033', '07044', '07054', '0706', '0709', '4560','4561','4562','570', '571', '5722','5723','5724','5725','5726','5727','5728','5733', '5734', '5738','5739', 'V427']
Pepticulcer_disease_9=['5317', '5319', '5327', '5329','5337', '5339', '5347', '5349']
AIDS_H1V_9=['042','043','044']
Lymphoma_9=['200','201','202','2030', '2386' ]
Metastatic_cancer_9=['196','197','198','199']
Solid_tumor_without_metastasis_9=['140','141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169','170', '171', '172', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195']
Rheumatoid_arthritis_collagen_vascular_diseases_9=['446', '7010', '7101','7102','7103','7104','7108', '7109', '7112', '714','7193', '720', '725', '7285','72889', '72930']
Coagulopathy_9=['286', '2871', '2873','2874','2875']
Obesity_9=['2780']
Weight_loss_9=['260','261','262','263','7832','7994']
Fluid_electrolyte_disorders_9=['2536', '276' ]
Blood_loss_anemia_9=['2800']
Deficiency_anemia_9=['2801','2802','2803','2804','2805','2806','2807','2808','2809', '281']
Alcohol_abuse_9=['2652', '2911','2912','2913','2915','2916','2917','2918','2919','3030','3039', '3050', '3575','4255', '5353','5710','5713', '980','V113']
Drug_abuse_9=['292','304','3052',  '3053',    '3054',    '3055',    '3056',    '3057',    '3058',    '3059','V6542'  ]
Psychoses_9=['2938', '295', '29604', '29614','29644','29654','297','298']
Depression_9=['2962', '2963', '2965', '3004','309', '311' ]

In [71]:
combined_list_9 = (hypertension_uncomplicated_9 +hypertension_complicated_9 +chf_9 +cardiac_9 +valvular_disease_9 +Pulmonary_circulation_9 +Peripheral_vascular_9 +Paralysis_9 +Other_neurological_disorders_9 +Chronic_pulmonary_disease_9 +Diabetes_uncomplicated_9 +Diabetes_complicated_9 +Hypothyroidism_9 +Renal_failure_9 +Liver_disease_9 +Pepticulcer_disease_9 +AIDS_H1V_9 +Lymphoma_9 +Metastatic_cancer_9 +Solid_tumor_without_metastasis_9 +Rheumatoid_arthritis_collagen_vascular_diseases_9 +Coagulopathy_9 +Obesity_9 +Weight_loss_9 +Fluid_electrolyte_disorders_9 +Blood_loss_anemia_9 +Deficiency_anemia_9 +Alcohol_abuse_9 +Drug_abuse_9 +Psychoses_9 +Depression_9)

In [72]:
hypertension_uncomplicated_10=['I10']
hypertension_complicated_10=['I11','I12','I13','I15']
chf_10=['I099', 'I110', 'I130','I13.2', 'I25.5', 'I42.0', 'I425','I426','I427','I428','I429','I43', 'I50', 'P290']
cardiac_10=['I441','I442','I443','I456','I459', 'I47', 'I48', 'I49', 'ROOO', 'ROO1', 'ROO8', 'T821', 'Z450', 'Z950']
valvular_disease_10=['A520','I05','I06','I07','I08','I091','I098', 'I34','I35','I36','I37','I38','I39', 'Q23O','Q231','Q232','Q233', 'Z952', 'Z954']
Pulmonary_circulation_10=['I26', 'I27', 'I280', 'I288', 'I289']
Peripheral_vascular_10=['I70', 'I71', 'I731', 'I738', 'I739', 'I771', 'I790', 'I792', 'K551', 'K558', 'K559', 'Z958', 'Z959' ]
Paralysis_10=['G041', 'G114', 'G801', 'G802', 'G81', 'G82','G830','G831','G832','G833','G834', 'G839' ]
Other_neurological_disorders_10=['G10','G11','G12','G13', 'G20','G21','G22', 'G254', 'G255', 'G312', 'G318', 'G319', 'G32', 'G35','G36','G37', 'G40', 'G41', 'G931', 'G934', 'R470', 'R56']
Chronic_pulmonary_disease_10=['I278', 'I279','J40','J41','J42','J43','J44','J45','J46','J47','J60','J61','J62','J63','J64','J65','J66','J67','J684', 'J701', 'J703' ]
Diabetes_uncomplicated_10=['E100', 'E101', 'E109', 'E110', 'E111', 'E119', 'E120', 'E121', 'E129', 'E130', 'E131','E139', 'E140', 'E141', 'E149' ]
Diabetes_complicated_10=['E102','E103','E104','E105','E106','E107','E108', 'E112','E113','E114','E115','E116','E117','E118','E122','E123','E124','E125','E126','E127','E128','E132','E133','E134','E135','E136''E137','E138','E142','E143','E144','E145','E146','E147','E148']
Hypothyroidism_10=['E00','E01','E02','E03','E890']
Renal_failure_10=['I120', 'I131', 'N18', 'NI9', 'N250', 'Z490', 'Z491', 'Z492','Z940', 'Z992' ]
Liver_disease_10=['B18', 'I85', 'I864', 'I982', 'K70', 'K711', 'K713','K715', 'K717', 'K72', 'K73', 'K74', 'K76','K762','K763','K764','K765','K766','K767','K768','K769','Z944']
Pepticulcer_disease_10=['K257', 'K259', 'K267', 'K269','K277', 'K279', 'K287', 'K289']
AIDS_H1V_10=['B20','B21','B22','B24']
Lymphoma_10=['C81','C82','C83','C84','C85', 'C88','C96', 'C90', 'C902']
Metastatic_cancer_10=['C77','C78','C79','C80']
Solid_tumor_without_metastasis_10=['C00','C01','C02','C03','C04','C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C30','C31','C32','C33','C34','C37','C38','C39','C40','C41','C43','C45','C46','C47','C48','C49','C50','C51','C52','C53','C54','C55','C56','C57','C58','C60','C61','C62','C63','C64','C65','C66','C67','C68','C69','C70','C71','C72','C73','C74',  'C75', 'C76','C97']
Rheumatoid_arthritis_collagen_vascular_diseases_10=['L940', 'L941', 'L943', 'M05','M06', 'M08', 'M120','M123', 'M30', 'M310','M311','M312','M313','M32','M33','M34','M35','M45','M461', 'M468','M469']
Coagulopathy_10=['D65','D66','D67','D68','D691','D693','D694','D695','D696']
Obesity_10=['E66']
Weight_loss_10=['E40','E41','E42','E43','E44','E45','E46','R634', 'R64']
Fluid_electrolyte_disorders_10=['E222','E86','E87']
Blood_loss_anemia_10=['D500']
Deficiency_anemia_10=['D508', 'D509', 'D51','D52','D53']
Alcohol_abuse_10=['F10', 'E52', 'G621', 'I426', 'K292', 'K700', 'K703','K709', 'T51', 'Z502','Z714', 'Z721' ]
Drug_abuse_10=['F11','F12','F13','F14','F15','F16','F18','F19','Z715','Z722' ]
Psychoses_10=['F20','F22','F23','F24','F25','F28','F29', 'F302', 'F312', 'F315']
Depression_10=['F204','F313','F314','F315', 'F32', 'F33', 'F341', 'F412', 'F432' ]

In [73]:
combined_list_10 = (hypertension_uncomplicated_10 +hypertension_complicated_10 +chf_10 +cardiac_10 +valvular_disease_10 +Pulmonary_circulation_10 +Peripheral_vascular_10 +Paralysis_10 +Other_neurological_disorders_10 +Chronic_pulmonary_disease_10 +Diabetes_uncomplicated_10 +Diabetes_complicated_10 +Hypothyroidism_10 +Renal_failure_10 +Liver_disease_10 +Pepticulcer_disease_10 +AIDS_H1V_10 +Lymphoma_10 +Metastatic_cancer_10 +Solid_tumor_without_metastasis_10 +Rheumatoid_arthritis_collagen_vascular_diseases_10 +Coagulopathy_10 +Obesity_10 +Weight_loss_10 +Fluid_electrolyte_disorders_10 +Blood_loss_anemia_10 +Deficiency_anemia_10 +Alcohol_abuse_10 +Drug_abuse_10 +Psychoses_10 +Depression_10)

In [74]:
icd_9_mask = (diag_df['icd_code'].str.startswith(tuple(combined_list_9)))&(diag_df['icd_version']==9)
icd_10_mask = (diag_df['icd_code'].str.startswith(tuple(combined_list_10)))&(diag_df['icd_version']==10)
icd_9_rows = diag_df[icd_9_mask]
icd_10_rows = diag_df[icd_10_mask]

In [75]:
diag_df_1=pd.concat([icd_9_rows,icd_10_rows])

In [76]:
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_uncomplicated_9)))&(diag_df['icd_version']==9), 'name'] = 'hypertension_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_complicated_9)))&(diag_df['icd_version']==9), 'name'] = 'hypertension_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(chf_9)))&(diag_df['icd_version']==9), 'name'] = 'chf'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(cardiac_9)))&(diag_df['icd_version']==9), 'name'] = 'cardiac'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(valvular_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'valvular_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pulmonary_circulation_9)))&(diag_df['icd_version']==9), 'name'] = 'Pulmonary_circulation'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Peripheral_vascular_9)))&(diag_df['icd_version']==9), 'name'] = 'Peripheral_vascular'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Paralysis_9)))&(diag_df['icd_version']==9), 'name'] = 'Paralysis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Other_neurological_disorders_9)))&(diag_df['icd_version']==9), 'name'] = 'Other_neurological_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Chronic_pulmonary_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Chronic_pulmonary_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_uncomplicated_9)))&(diag_df['icd_version']==9), 'name'] = 'Diabetes_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_complicated_9)))&(diag_df['icd_version']==9), 'name'] = 'Diabetes_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Hypothyroidism_9)))&(diag_df['icd_version']==9), 'name'] = 'Hypothyroidism'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Renal_failure_9)))&(diag_df['icd_version']==9), 'name'] = 'Renal_failure'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Liver_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Liver_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pepticulcer_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Pepticulcer_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(AIDS_H1V_9)))&(diag_df['icd_version']==9), 'name'] = 'AIDS_H1V'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Lymphoma_9)))&(diag_df['icd_version']==9), 'name'] = 'Lymphoma'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Metastatic_cancer_9)))&(diag_df['icd_version']==9), 'name'] = 'Metastatic_cancer'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Solid_tumor_without_metastasis_9)))&(diag_df['icd_version']==9), 'name'] = 'Solid_tumor_without_metastasis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Rheumatoid_arthritis_collagen_vascular_diseases_9)))&(diag_df['icd_version']==9), 'name'] = 'Rheumatoid_arthritis_collagen_vascular_diseases'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Coagulopathy_9)))&(diag_df['icd_version']==9), 'name'] = 'Coagulopathy'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Obesity_9)))&(diag_df['icd_version']==9), 'name'] = 'Obesity'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Weight_loss_9)))&(diag_df['icd_version']==9), 'name'] = 'Weight_loss'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Fluid_electrolyte_disorders_9)))&(diag_df['icd_version']==9), 'name'] = 'Fluid_electrolyte_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Blood_loss_anemia_9)))&(diag_df['icd_version']==9), 'name'] = 'Blood_loss_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Deficiency_anemia_9)))&(diag_df['icd_version']==9), 'name'] = 'Deficiency_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Alcohol_abuse_9)))&(diag_df['icd_version']==9), 'name'] = 'Alcohol_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Drug_abuse_9)))&(diag_df['icd_version']==9), 'name'] = 'Drug_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Psychoses_9)))&(diag_df['icd_version']==9), 'name'] = 'Psychoses'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Depression_9)))&(diag_df['icd_version']==9), 'name'] = 'Depression'

In [77]:
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_uncomplicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'hypertension_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_complicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'hypertension_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(chf_10)))&(diag_df['icd_version']== 10), 'name'] = 'chf'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(cardiac_10)))&(diag_df['icd_version']== 10), 'name'] = 'cardiac'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(valvular_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'valvular_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pulmonary_circulation_10)))&(diag_df['icd_version']== 10), 'name'] = 'Pulmonary_circulation'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Peripheral_vascular_10)))&(diag_df['icd_version']== 10), 'name'] = 'Peripheral_vascular'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Paralysis_10)))&(diag_df['icd_version']== 10), 'name'] = 'Paralysis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Other_neurological_disorders_10)))&(diag_df['icd_version']== 10), 'name'] = 'Other_neurological_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Chronic_pulmonary_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Chronic_pulmonary_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_uncomplicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'Diabetes_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_complicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'Diabetes_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Hypothyroidism_10)))&(diag_df['icd_version']== 10), 'name'] = 'Hypothyroidism'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Renal_failure_10)))&(diag_df['icd_version']== 10), 'name'] = 'Renal_failure'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Liver_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Liver_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pepticulcer_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Pepticulcer_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(AIDS_H1V_10)))&(diag_df['icd_version']== 10), 'name'] = 'AIDS_H1V'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Lymphoma_10)))&(diag_df['icd_version']== 10), 'name'] = 'Lymphoma'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Metastatic_cancer_10)))&(diag_df['icd_version']== 10), 'name'] = 'Metastatic_cancer'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Solid_tumor_without_metastasis_10)))&(diag_df['icd_version']== 10), 'name'] = 'Solid_tumor_without_metastasis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Rheumatoid_arthritis_collagen_vascular_diseases_10)))&(diag_df['icd_version']== 10), 'name'] = 'Rheumatoid_arthritis_collagen_vascular_diseases'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Coagulopathy_10)))&(diag_df['icd_version']== 10), 'name'] = 'Coagulopathy'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Obesity_10)))&(diag_df['icd_version']== 10), 'name'] = 'Obesity'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Weight_loss_10)))&(diag_df['icd_version']== 10), 'name'] = 'Weight_loss'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Fluid_electrolyte_disorders_10)))&(diag_df['icd_version']== 10), 'name'] = 'Fluid_electrolyte_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Blood_loss_anemia_10)))&(diag_df['icd_version']== 10), 'name'] = 'Blood_loss_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Deficiency_anemia_10)))&(diag_df['icd_version']== 10), 'name'] = 'Deficiency_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Alcohol_abuse_10)))&(diag_df['icd_version']== 10), 'name'] = 'Alcohol_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Drug_abuse_10)))&(diag_df['icd_version']== 10), 'name'] = 'Drug_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Psychoses_10)))&(diag_df['icd_version']== 10), 'name'] = 'Psychoses'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Depression_10)))&(diag_df['icd_version']== 10), 'name'] = 'Depression'

In [78]:
diag_df_1

,stay_id,icd_code,icd_version,name
3,33258284,5728,9,Liver_disease
7,35968195,5715,9,Liver_disease
12,38112554,5715,9,Liver_disease
15,39399961,34830,9,Other_neurological_disorders
35,35106839,2860,9,Coagulopathy
...,...,...,...,...
898949,39259698,E119,10,Diabetes_uncomplicated
898951,39259698,I10,10,hypertension_uncomplicated
899010,35847469,I10,10,hypertension_uncomplicated
899029,30402875,F29,10,Psychoses


In [79]:
diag_df_2=diag_df_1[diag_df_1['stay_id'].isin(df_3['stay_id'])]
diag_df_2=diag_df_2[['stay_id','name']]

In [80]:
diag_df_2

,stay_id,name
7,35968195,Liver_disease
12,38112554,Liver_disease
15,39399961,Other_neurological_disorders
71,37059738,Solid_tumor_without_metastasis
72,37290428,Depression
...,...,...
898939,36522433,Diabetes_uncomplicated
898949,39259698,Diabetes_uncomplicated
898951,39259698,hypertension_uncomplicated
899010,35847469,hypertension_uncomplicated


In [81]:
diag_df_2.rename(columns={'name': 'Comorbidity'}, inplace=True)
one_hot_diag_df_2 = pd.get_dummies(diag_df_2, columns=['Comorbidity']).astype(int)
#one_hot_diag_df_2.to_csv('one_hot_diag.csv')

In [82]:
one_hot_diag_df_2

,stay_id,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
7,35968195,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,38112554,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,39399961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71,37059738,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
72,37290428,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898939,36522433,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
898949,39259698,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
898951,39259698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
899010,35847469,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [83]:
one_hot_diag_df_2=one_hot_diag_df_2.sort_values(by='stay_id')

In [84]:
one_hot_diag_df_2.reset_index(drop=True,inplace=True)

In [85]:
one_hot_diag_df_2

,stay_id,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,30000094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,30000094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,30000112,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136815,39999835,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
136816,39999863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
136817,39999933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
136818,39999933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [86]:
diag_df_4=one_hot_diag_df_2.groupby('stay_id').max().reset_index()

In [87]:
#diag_df_4.to_csv("diag_stay_id_231220.csv",index=0)

In [88]:
diag_df_4

,stay_id,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,30000094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,30000112,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,30000177,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102100,39999734,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
102101,39999835,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
102102,39999863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
102103,39999933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
df_4=pd.merge(df_3, diag_df_4,on='stay_id',how='left')

In [90]:
df_4

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411025,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,21

In [91]:
df_4.iloc[:, 30:]

,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1411026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1411027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1411028,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
df_4.iloc[:, 30:] = df_4.iloc[:, 30:].fillna(0)

In [93]:
df_4.reset_index(inplace=True,drop=True)

In [94]:
df_4

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411025,39999953,2152-06-22 18:13:00,F,Other,WALK IN,ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-06-22 14:08:00,21

In [95]:
vital=pd.read_parquet('vitalsign.parquet')
vital_df=vital[['stay_id','charttime','temperature','heartrate','resprate','o2sat','sbp','dbp']]
vital_df.rename(columns={'charttime': 'time'}, inplace=True)
vital_df

/tmp/ipykernel_2449120/745313819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vital_df.rename(columns={'charttime': 'time'}, inplace=True)


,stay_id,time,temperature,heartrate,resprate,o2sat,sbp,dbp
0,32952584,2180-07-22 16:36:00,NaN,83.0,24.0,97.0,90.0,51.0
1,32952584,2180-07-22 16:43:00,NaN,85.0,22.0,98.0,76.0,39.0
2,32952584,2180-07-22 16:45:00,NaN,84.0,22.0,97.0,75.0,39.0
3,32952584,2180-07-22 17:56:00,NaN,84.0,20.0,99.0,86.0,51.0
4,32952584,2180-07-22 18:37:00,98.4,86.0,20.0,98.0,65.0,37.0
...,...,...,...,...,...,...,...,...
1564605,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0
1564606,32002659,2158-12-24 11:43:00,99.5,81.0,10.0,100.0,93.0,55.0
1564607,34731548,2145-11-02 19:40:00,NaN,112.0,18.0,NaN,118.0,83.0
1564608,34731548,2145-11-02 20:11:00,NaN,111.0,18.0,NaN,123.0,82.0


In [96]:
vital_df=vital_df[vital_df['stay_id'].isin(df_4['stay_id'])]
vital_df

,stay_id,time,temperature,heartrate,resprate,o2sat,sbp,dbp
8,35968195,2180-08-05 23:50:00,98.5,96.0,17.0,100.0,102.0,58.0
9,35968195,2180-08-06 01:07:00,98.1,91.0,18.0,99.0,98.0,60.0
10,38112554,2180-06-26 18:42:00,97.9,76.0,18.0,95.0,95.0,64.0
11,38112554,2180-06-26 20:54:00,97.9,86.0,17.0,93.0,96.0,57.0
12,39399961,2180-07-23 07:19:00,NaN,79.0,18.0,93.0,86.0,57.0
...,...,...,...,...,...,...,...,...
1564604,32917002,2149-01-08 16:00:00,98.1,102.0,17.0,96.0,121.0,71.0
1564605,32917002,2149-01-08 17:10:00,98.1,109.0,15.0,96.0,111.0,78.0
1564607,34731548,2145-11-02 19:40:00,NaN,112.0,18.0,NaN,118.0,83.0
1564608,34731548,2145-11-02 20:11:00,NaN,111.0,18.0,NaN,123.0,82.0


In [97]:
vital_df.reset_index(inplace=True,drop=True)

In [98]:
vital_df['temperature'] = (vital_df['temperature'] - 32) / 1.8
vital_df['temperature'] = vital_df['temperature'].apply(lambda x: np.nan if x < 32 or x >= 43 else x)
vital_df['heartrate'] = vital_df['heartrate'].apply(lambda x: np.nan if x <= 0 or x >= 300 else x)
vital_df['o2sat'] = vital_df['o2sat'].apply(lambda x: np.nan if x < 10 or x > 100 else x)
vital_df['resprate'] = vital_df['resprate'].apply(lambda x: np.nan if x <= 0 or x > 60 else x)
vital_df['sbp'] = vital_df['sbp'].apply(lambda x: np.nan if x <= 0 or x > 300 else x)
vital_df['dbp'] = vital_df['dbp'].apply(lambda x: np.nan if x < 10 or x > 175 else x)
vital_df.loc[vital_df['sbp'] < vital_df['dbp'], ['sbp', 'dbp']] = np.nan

/tmp/ipykernel_2449120/1419221124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vital_df['temperature'] = (vital_df['temperature'] - 32) / 1.8
/tmp/ipykernel_2449120/1419221124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vital_df['temperature'] = vital_df['temperature'].apply(lambda x: np.nan if x < 32 or x >= 43 else x)
/tmp/ipykernel_2449120/1419221124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [99]:
vital_df=vital_df.sort_values(by=['stay_id', 'time'])
vital_df.reset_index(drop=True,inplace=True)

In [100]:
vital_df

,stay_id,time,temperature,heartrate,resprate,o2sat,sbp,dbp
0,30000012,2126-02-14 20:22:00,37.111111,96.0,18.0,93.0,160.0,54.0
1,30000012,2126-02-14 23:43:00,NaN,80.0,13.0,99.0,112.0,44.0
2,30000012,2126-02-15 00:50:00,37.000000,88.0,16.0,100.0,135.0,51.0
3,30000038,2152-12-07 16:38:00,36.166667,54.0,18.0,95.0,143.0,73.0
4,30000038,2152-12-07 19:17:00,NaN,78.0,20.0,96.0,141.0,61.0
...,...,...,...,...,...,...,...,...
1163939,39999953,2152-06-22 17:30:00,36.833333,81.0,16.0,100.0,124.0,86.0
1163940,39999961,2145-05-16 17:17:00,37.388889,119.0,22.0,NaN,132.0,74.0
1163941,39999961,2145-05-16 19:56:00,NaN,117.0,20.0,100.0,128.0,77.0
1163942,39999961,2145-05-16 22:47:00,36.666667,107.0,20.0,99.0,139.0,90.0


In [101]:
df_5=pd.concat([df_4,vital_df])
df_5

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30000012,2126-02-15 01:22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-15 01:59:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30000038,2152-12-07 16:37:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2152-12-07 16:37:00,2152-12-07 19:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163939,39999953,2152-06-22 17:30:00,NaN,NaN,NaN,NaN,36.833333,81.0,16.0,100.0,124.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,

In [102]:
df_5['time']=pd.to_datetime(df_5['time'])
df_5=df_5.sort_values(by=['stay_id', 'time'])
df_5.reset_index(drop=True,inplace=True)

In [103]:
df_5

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:22:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-14 23:43:00,NaN,NaN,NaN,NaN,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 00:50:00,NaN,NaN,NaN,NaN,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574969,39999961,2145-05-16 19:56:00,NaN,NaN,NaN,NaN,NaN,117.0,20.0,100.0,128.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [104]:
df_5['first_time']=df_5.groupby('stay_id')['first_time'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df_5['last_time']=df_5.groupby('stay_id')['last_time'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [105]:
df_5

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:22:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-14 23:43:00,NaN,NaN,NaN,NaN,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 00:50:00,NaN,NaN,NaN,NaN,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574969,39999961,2145-05-16 19:56:00,NaN,NaN,NaN,NaN,NaN,117.0,20.0,100.0,128.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [106]:
df_6=df_5[df_5['last_time'] >= df_5['time']]

In [107]:
df_6

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:22:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-14 23:43:00,NaN,NaN,NaN,NaN,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 00:50:00,NaN,NaN,NaN,NaN,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574969,39999961,2145-05-16 19:56:00,NaN,NaN,NaN,NaN,NaN,117.0,20.0,100.0,128.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [108]:
df_7=df_6[df_6['first_time'] <= df_6['time']]

In [109]:
df_7

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:22:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-14 23:43:00,NaN,NaN,NaN,NaN,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 00:50:00,NaN,NaN,NaN,NaN,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574969,39999961,2145-05-16 19:56:00,NaN,NaN,NaN,NaN,NaN,117.0,20.0,100.0,128.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [110]:
vital_df['stay_id'].nunique()

330579

In [111]:
df_8=df_7[df_7['stay_id'].isin(vital_df['stay_id'].unique())]

In [112]:
df_8.reset_index(drop=True,inplace=True)

In [113]:
df_8

,stay_id,time,gender,race,arrival_transport,result,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,chief_abdpain,chief_transfer,chief_chestpain,chief_pfall,chief_dyspnea,chief_fever,chief_headache,chief_backpain,chief_weakness,chief_dizziness,chief_cough,chief_woundeval,chief_mvc,chief_etoh,chief_altered_mental_status,first_time,last_time,drug_acetaminophen,drug_morphine,drug_ondansetron,drug_hydromorphone,drug_oxycodone,drug_lidocaine,drug_ibuprofen,drug_lorazepam,drug_albuterol,drug_ketorolac,drug_ceftriaxone,drug_ipratropium,drug_vancomycin,drug_bromide,drug_diazepam,drug_aspirin,drug_sodium,drug_magnesium,drug_fentanyl,drug_diphenhydramine,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,30000012,2126-02-14 20:22:00,F,White,AMBULANCE,hos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30000012,2126-02-14 20:22:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30000012,2126-02-14 20:23:00,NaN,NaN,NaN,NaN,37.111111,96.0,18.0,93.0,160.0,54.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2126-02-14 20:22:00,2126-02-15 01:59:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30000012,2126-02-14 23:43:00,NaN,NaN,NaN,NaN,NaN,80.0,13.0,99.0,112.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30000012,2126-02-15 00:50:00,NaN,NaN,NaN,NaN,37.000000,88.0,16.0,100.0,135.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2126-02-14 20:22:00,2126-02-15 01:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2422686,39999961,2145-05-16 19:56:00,NaN,NaN,NaN,NaN,NaN,117.0,20.0,100.0,128.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [114]:
df_8.to_csv("pre_ED_240308.csv",index=0)

In [115]:
df_8.to_parquet("pre_ED_240308.parquet",index=0)